In [2]:
import os
import gym
import gym_donkeycar
import numpy as np
import random
import tensorflow
import shimmy
from rich.live import Live
import stable_baselines3
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import TD3

In [3]:
DONKEY_GYM = True
DONKEY_SIM_PATH = "remote"
DONKEY_GYM_ENV_NAME = "donkey-generated-track-v0"
body_style = "car01"
body_rgb = (128, 128, 128)
car_name = "TD32"
font_size = 100
WEB_CONTROL_PORT = int(os.getenv("WEB_CONTROL_PORT", 8887))
port = 9091

In [4]:
folder = '/mnt/c/Users/user/OneDrive/Desktop/DonkeyCar/TD32/'

In [5]:
conf = { "DONKEY_GYM":DONKEY_GYM, "DONKEY_SIM_PATH":DONKEY_SIM_PATH, "DONKEY_GYM_ENV_NAME":DONKEY_GYM_ENV_NAME, 
         "body_style":body_style, "body_rgb":body_rgb, "car_name":car_name, "font_size":font_size, 
         "WEB_CONTROL_PORT" : WEB_CONTROL_PORT, "port" : port }

In [6]:
env = gym.make("donkey-minimonaco-track-v0", conf=conf)

INFO:gym_donkeycar.core.client:connecting to localhost:9091 
/home/user/miniconda3/envs/donkey/lib/python3.11/site-packages/gym/spaces/box.py:78: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


starting DonkeyGym env
Setting default: start_delay 5.0
Setting default: max_cte 8.0
Setting default: frame_skip 1
Setting default: cam_resolution (120, 160, 3)
Setting default: log_level 20
Setting default: host localhost
Setting default: steer_limit 1.0
Setting default: throttle_min 0.0
Setting default: throttle_max 1.0
loading scene mini_monaco


INFO:gym_donkeycar.envs.donkey_sim:on need car config
INFO:gym_donkeycar.envs.donkey_sim:sending car config.
INFO:gym_donkeycar.envs.donkey_sim:sim started!


In [10]:
for i in range(1000):
    if i == 0:
        model = TD3("CnnPolicy", env, verbose = 0, buffer_size=10000, learning_rate = 0.00000005 )
        Log={"TestReward":[]}

    # training
    obs = env.reset()
    model.learn(total_timesteps=2500, progress_bar=0)
    model.save(f'{folder}/{(i+1)*2500}.pth')

    # testing
    tem = 0
    for _ in range(20):
        obs = env.reset()
        while True:
            action, _states = model.predict(obs, deterministic=True)
            obs, reward, done, info = env.step(action)
            # print (reward, done)
            env.render()
            tem += reward
            if done: break
    Log["TestReward"].append(tem/20)
    print(f"testing {(i+1)*2500} : {tem/20}")
    np.save(f"{folder}Log_TD32-{(i+1)*2500}.npy", Log)

testing 2500 : 65.79229832808642
testing 5000 : 62.75394494547896
testing 7500 : 57.578114083700385
testing 10000 : 52.10104304024126
testing 12500 : 47.937331435784124
testing 15000 : 44.5316900745156
testing 17500 : 40.82724596199314
testing 20000 : 38.356784185950744
testing 22500 : 36.01564048185263
testing 25000 : 32.49153717712064
testing 27500 : 28.87336971522682
testing 30000 : 26.64719340054209
testing 32500 : 24.235018371247897
testing 35000 : 22.74848323459005
testing 37500 : 21.283274368114988
testing 40000 : 19.947333499654167
testing 42500 : 18.963767588652697
testing 45000 : 18.11184692667192
testing 47500 : 17.720746347278997
testing 50000 : 17.0789630459929
testing 52500 : 16.70835363681484
testing 55000 : 16.656893849204035
testing 57500 : 16.692804742638295
testing 60000 : 16.625846864419945
testing 62500 : 16.646772526285602
testing 65000 : 16.7112560429675
testing 67500 : 16.563042394447056
testing 70000 : 16.724437487385845
testing 72500 : 17.177712751654056
testi

KeyboardInterrupt: 

In [34]:
num = 2500
test_model = TD3.load(f"{folder}{num}.pth")
obs = env.reset()
tem = 0
while True:
    action, _states = test_model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    print (reward, done)
    env.render()
    tem += reward
    if done: break
print(tem)

In [11]:
env.close()